In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install easyocr
!apt-get install poppler-utils 
!pip install PyMuPDF
!sudo python -m nltk.downloader all


In [ ]:
!which tesseract

/usr/bin/tesseract


In [ ]:
import pytesseract
from PIL import Image
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#from pdf2image import convert_from_path, convert_from_bytes
import os
import tempfile
#from pdf2image import convert_from_path
#import poppler
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image


In [ ]:
import pytesseract
from PIL import Image
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import tempfile
import cv2
import numpy as np

In [ ]:
import cv2
import numpy as np

#img = cv2.imread('image1_1.jpg')

# get grayscale image
def get_grayscale(image):
  return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
  return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
  return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
  kernel = np.ones((5,5),np.uint8)
  return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
  kernel = np.ones((5,5),np.uint8)
  return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
  kernel = np.ones((5,5),np.uint8)
  return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
  return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
  coords = np.column_stack(np.where(image > 0))
  angle = cv2.minAreaRect(coords)[-1]
  if angle < -45:
    angle = -(90 + angle)
  else:
    angle = -angle
  (h, w) = image.shape[:2]
  center = (w // 2, h // 2)
  M = cv2.getRotationMatrix2D(center, angle, 1.0)
  rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
  return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

In [ ]:
def flattenList(nestedList):
  
    # check if list is empty
    if not(bool(nestedList)):
        return nestedList
  
     # to check instance of list is empty or not
    if isinstance(nestedList[0], list):
  
        # call function with sublist as argument
        return flattenList(*nestedList[:1]) + flattenList(nestedList[1:])
  
    # call function with sublist as argument
    return nestedList[:1] + flattenList(nestedList[1:])
  

In [ ]:
## Pytesseract Method

import fitz
import io
from PIL import Image
import pandas as pd
import pytesseract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import re
import tempfile
import cv2
import numpy as np
from tqdm import tqdm

dataset = '/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Test'
df_train = pd.read_csv('/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Train.csv', sep=',', header=0)
df_test = pd.read_csv('/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Test.csv', sep=',', header=0)
output = []

for files in tqdm(os.listdir(dataset)):
  file_name=os.path.join(dataset, files)
  pdf_file = fitz.open(file_name)
  for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.getImageList()
    for image_index, img in enumerate(page.getImageList(), start=1):
      xref = img[0]
      base_image = pdf_file.extractImage(xref)
      image_bytes = base_image["image"]
      image_ext = base_image["ext"]
      image = Image.open(io.BytesIO(image_bytes))
      image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb")) 

  
  #img = Image.open('image1_1.jpeg').convert('L')
  img = get_grayscale(cv2.imread('image1_1.jpeg'))
  text = pytesseract.image_to_string(img)
  processed_text = text.lower()
  processed_text = processed_text.replace('\n', ' ')
  processed_text = processed_text.replace('*', ' ')
  text_tokens = word_tokenize(processed_text)
  text_without_stopwords = [word for word in text_tokens if not word in stopwords.words()]
  processed_text = " ".join(text_without_stopwords)
  text_list = word_tokenize(processed_text)
  Total_Amount_List = []
  for index in range (len(text_list)):
    if text_list[index] == 'total' or text_list[index] == 'tot' or text_list[index] == 'tctal' or text_list[index] == 'tct':
      try:
        Total_Amount_List.append(text_list[index+1])
        Total_Amount_List.append(text_list[index+2])
      except IndexError:
        Total_Amount_List.append(0.0)


  print(Total_Amount_List)
  Total_Amount = [] 
  for item in Total_Amount_List:
    try:
      amt = [float(s) for s in re.findall(r'-?\d+\.?\d*', item)]
      Total_Amount.append(amt)
    except TypeError:
      pass
  Total_Amount = flattenList(Total_Amount)
  try:
    output.append(max(Total_Amount))
  except ValueError:
    output.append(0.0)

df_test["Predicted Amt 2"] = output
df_train.head()



'''
  Total_Amount_List = []
  for index in range (len(text_list)):
    if text_list[index] == 'total' or text_list[index] == 'tot':
      try:
        Total_Amount_List.append(text_list[index+1])
        Total_Amount_List.append(text_list[index+2])
      except IndexError:
        Total_Amount_List.append(0.0)


  print(Total_Amount_List)
  Total_Amount = [] 
  for item in Total_Amount_List:
    try:
      Total_Amount.append(float(item))
    except ValueError:
      pass
  try:
    output.append(max(Total_Amount))
  except ValueError:
    output.append(0.0)

df_test["Predicted Amt 2"] = output
df_train.head()
'''



  1%|▏         | 1/70 [00:01<01:21,  1.19s/it]

[0.0]


  3%|▎         | 2/70 [00:02<01:28,  1.30s/it]

[]


  4%|▍         | 3/70 [00:03<01:24,  1.25s/it]

...

 99%|█████████▊| 69/70 [01:35<00:01,  1.14s/it]

['tip', ':']


100%|██████████| 70/70 [01:36<00:00,  1.39s/it]

['20.62', 'seat', '13.13', 'subtotal', '33.15', 'balance']


'\n  Total_Amount_List = []\n  for index in range (len(text_list)):\n    if text_list[index] == \'total\' or text_list[index] == \'tot\':\n      try:\n        Total_Amount_List.append(text_list[index+1])\n        Total_Amount_List.append(text_list[index+2])\n      except IndexError:\n        Total_Amount_List.append(0.0)\n\n\n  print(Total_Amount_List)\n  Total_Amount = [] \n  for item in Total_Amount_List:\n    try:\n      Total_Amount.append(float(item))\n    except ValueError:\n      pass\n  try:\n    output.append(max(Total_Amount))\n  except ValueError:\n    output.append(0.0)\n\ndf_test["Predicted Amt 2"] = output\ndf_train.head()\n'

In [ ]:
df_test.head()

,FileName,Total Amount,Predicted Amt 2
0,TE_131,NaN,0.00
1,TE_132,NaN,0.00
2,TE_133,NaN,0.00
3,TE_134,NaN,47.09
4,TE_135,NaN,47.09


In [ ]:
## Easy-OCR Method

import fitz
import io
from PIL import Image
import pandas as pd
import easyocr
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import tempfile
import cv2
import numpy as np
from tqdm import tqdm

dataset = '/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Test'
df_train = pd.read_csv('/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Train.csv', sep=',', header=0)
df_test = pd.read_csv('/content/drive/MyDrive/CTRL_HACK_DEL/Final_Test_Q4/Dataset_test/Test.csv', sep=',', header=0)
output = []

for files in tqdm(os.listdir(dataset)):
  file_name=os.path.join(dataset, files)
  pdf_file = fitz.open(file_name)
  for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.getImageList()
    for image_index, img in enumerate(page.getImageList(), start=1):
      xref = img[0]
      base_image = pdf_file.extractImage(xref)
      image_bytes = base_image["image"]
      image_ext = base_image["ext"]
      image = Image.open(io.BytesIO(image_bytes))
      image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb")) 

  
  #img = Image.open('image1_1.jpeg').convert('L')
  #img = get_grayscale(cv2.imread('image1_1.jpeg'))
  #text = pytesseract.image_to_string(img)
  reader = easyocr.Reader(['en'])
  text = reader.readtext('image1_1.jpeg', detail = 0)
  #text_list = text

  #processed_text = text.lower()
  #processed_text = processed_text.replace('\n', ' ')
  #processed_text = processed_text.replace('*', ' ')
  #text_tokens = word_tokenize(processed_text)
  #text_without_stopwords = [word for word in text_tokens if not word in stopwords.words()]
  #processed_text = " ".join(text_without_stopwords)
  #text_list = word_tokenize(processed_text)
  text_list = [x.lower() for x in text]
  Total_Amount_List = []
  for index in range (len(text_list)):
    if text_list[index] == 'total' or text_list[index] == 'tot' or text_list[index] == 'tctal' or text_list[index] == 'tct':
      try:
        Total_Amount_List.append(text_list[index+1])
        Total_Amount_List.append(text_list[index+2])
      except IndexError:
        Total_Amount_List.append(0.0)


  print(Total_Amount_List)
  Total_Amount = [] 
  for item in Total_Amount_List:
    try:
      amt = [float(s) for s in re.findall(r'-?\d+\.?\d*', item)]
      Total_Amount.append(amt)
    except TypeError:
      pass
  Total_Amount = flattenList(Total_Amount)
  try:
    output.append(max(Total_Amount))
  except ValueError:
    output.append(0.0)

df_test["Predicted Amt 2"] = output
df_train.head()




  0%|          | 0/70 [00:00<?, ?it/s]


  1%|▏         | 1/70 [00:04<05:18,  4.62s/it]

['61', '216']





  3%|▎         | 2/70 [00:09<05:11,  4.59s/it]

[]





  4%|▍         | 3/70 [00:13<05:07,  4.59s/it]

...




 99%|█████████▊| 69/70 [05:06<00:04,  4.16s/it]

[]





100%|██████████| 70/70 [05:11<00:00,  4.45s/it]

['13,13', 'subtotal', '33.75', 'ba tance']


,FileName,Total Amount
0,TR_1,56.58
1,TR_2,69.25
2,TR_3,7.61
3,TR_4,5.35
4,TR_5,15.03


In [ ]:
df_test.to_csv('Q4_submission_pytesseract_1.csv', header=True, index=False)

In [ ]:
import fitz
import io
from PIL import Image
import pandas as pd
import easyocr
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os
import tempfile
import cv2
import numpy as np
import re
from tqdm import tqdm

output = []
reader = easyocr.Reader(['en'])
text = reader.readtext('image1_1.jpeg', detail = 0)
print(text)
text_list = text
#processed_text = text.lower()
#processed_text = processed_text.replace('\n', ' ')
#processed_text = processed_text.replace('*', ' ')
#text_tokens = word_tokenize(processed_text)
#text_without_stopwords = [word for word in text_tokens if not word in stopwords.words()]
#processed_text = " ".join(text_without_stopwords)
#text_list = word_tokenize(processed_text)
text_list = [x.lower() for x in text]
Total_Amount_List = []
for index in range (len(text_list)):
  if text_list[index] == 'total' or text_list[index] == 'tot':
    try:
      Total_Amount_List.append(text_list[index+1])
      Total_Amount_List.append(text_list[index+2])
    except IndexError:
      Total_Amount_List.append(0.0)


print(Total_Amount_List)
#Total_Amount_List = [float(s) for s in re.findall(r'-?\d+\.?\d*', 'he33.45llo -42 I\'m a 32 string 30')]
Total_Amount = [] 
for item in Total_Amount_List:
  try:
    amt = [float(s) for s in re.findall(r'-?\d+\.?\d*', item)]
    Total_Amount.append(amt)
  except ValueError:
    pass
print(Total_Amount)
Total_Amount = flattenList(Total_Amount)
print(Total_Amount)
try:
  output.append(max(Total_Amount))
except ValueError:
  output.append(0.0)

print(max(Total_Amount))
print(output)

['Amici Conyers', '1805 ParkerRD Suite G11O', '18C5 Parker RD', '(678)   890-2005', 'Server : Nicole', '06/10/2018', 'Table 52/1', '1;02 PM', 'Guests:', 'Seat', '{+', 'Coke', 'Margherita', 'Ca izcne', '2.29', '11.99', 'Side Caesar', '4,99', 'Subtctal', '19,27', 'Tax', '1,35', 'Tcta]', '20,62', 'Seat 2', 'Krtr*', 'Coke', '2,29', 'Db/ Stack Cheeseturger', '8,99', 'Bacon', '0.99', 'Subtcta]', '12,27', 'Tax', '0.86', 'Tctal', '13,13', 'Subtotal', '31,54', 'Tax', '2,21', 'Total', '33.75', 'Ba Tance', 'DJue', '33', '75', 'Thank you for dining with us!', 'SUGGE:TED GRATUITY:', '30004']
['33.75', 'ba tance']
[[33.75], []]
[33.75]
33.75
[33.75]
